# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('idf_efd')
start = "2023-07-06T23:00:00"
end = "2023-07-07T07:30:00"
start = "2023-07-04T06:32:00"
end = "2023-07-04T06:35:00"
start = "2023-07-04T04:50:00"
end = "2023-07-04T04:58:00"

bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))

## Plot the Bump Test states

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

id=330
this_bump = bumps[bumps['actuatorId']==id]
last_this_bump_index = bumps[bumps['actuatorId']==id].last_valid_index()
pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
fa = force_actuator_from_id(id)
primary_bump = f"primaryTest{fa.index}"
fig, ax = plt.subplots(1,1)
this_bump[primary_bump].plot(ax=ax)
timestamp = bumps.index[0].isoformat().split('.')[0]
ax. set_title(f" Bump Test States id = {id} {timestamp}")
ax.set_ylim(0,8)
ax2 = ax.twinx()
ax2.set_yticks([0,1,2,3,4,5,6,7,8])
ax2.set_yticklabels(['', 'NOTTESTED','TESTINGPOSITIVE','TESTINGPOSITIVEWAIT', \
                   'TESTINGNEGATIVE', 'TESTINGNEGATIVEWAIT','PASSED','FAILED', ''])
#plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_States.png")

if pass_fail[primary_bump] == 6:
    print(f"Primary {id} passed the bump test")
elif pass_fail[primary_bump] == 7:
    print(f"Primary {id} failed the bump test")
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_States_{id}_{timestamp}.png")